# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [7]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [8]:
df = pd.read_csv('data.csv')
df.info()
df.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**
Рассмотрим полученные данные подробее:
В таблице имеется 12 столбцов, тип данных у каждого:

* children - int64
* days_employed - float64
* dob_years - int64
* education - object
* education_id - int64
* family_status - object
* family_status_id - int64
* gender - object
* income_type - object
* debt - int64
* total_income - float64
* purpose - object

Подробнее разберем какие в *df* столбцы и какую информацию они несут:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Количество значений в столбцах отличается, это означает, что есть пропущенные значения в стоблцах:
*days_employed*,*total_income*.
В столбце *childred* мы обнаружели несколько артефактов: отрицательное кол-во детей и кол-во детей 20. Вероятнее всего данные артефакты образовались в следствии того, что респонденты не указали данные и для явного выбивания из списка их пометили таким образом. ПОэтому мы присвоили им значения нуль, так как их немного и их значениями для исследования можно пренебречь.
В столбце *days_emploed* также есть артефакты, такие как отрицательный стаж и невожножный стаж работы. С этими артефактами мы разберемся во втором шаге. Есть еще один артефакт в столбце *gender* - **XNA**. Так как для нашего исследования пол респондентов не несет информативности им мы можем пренебречь

## Шаг 2. Предобработка данных

### Обработка пропусков

In [9]:
days_employed_median = df.loc[:15905, 'days_employed'].median()
total_income_median = df.sort_values(by = 'total_income') \
    .reset_index(drop = True) \
    .loc[:19351, 'total_income'].median()

df['days_employed'] = df['days_employed'].fillna(days_employed_median)

for row in range(len(df)):
    if df.loc[row, 'days_employed'] > 0:
        if df.loc[row, 'gender'] == 'M':
            df.loc[row, 'days_employed'] = 15147
        if df.loc[row, 'gender'] == 'F':
            df.loc[row, 'days_employed'] = 13322
        if abs(df.loc[row, 'days_employed']/365) > df.loc[row, 'dob_years']:
            df.loc[row, 'days_employed'] = days_employed_median
df['days_employed'] = abs(df['days_employed'])
total_income_group = df.groupby('income_type')['total_income'].median()

maps = {'безработный': 131339.75,'в декрете': 53829.13, 'госслужащий': 150447.93, 'компаньон': 172357.95, 
        'пенсионер': 118514.48, 'предприниматель': 499163.14, 'сотрудник': 142594.39, 'студент': 98201.62}
df['total_income'] = df['total_income'].fillna(-1)

for row in range(len(df)):
    if df.loc[row, 'total_income'] == -1:
        df.loc[row, 'total_income'] = maps[df.loc[row, 'income_type']]

print(maps)

{'безработный': 131339.75, 'в декрете': 53829.13, 'госслужащий': 150447.93, 'компаньон': 172357.95, 'пенсионер': 118514.48, 'предприниматель': 499163.14, 'сотрудник': 142594.39, 'студент': 98201.62}


Значение, которым будем заполнять пропуски если потенциальный заёмщик пенсионер, то такое медианное значение стажа ему не подойдет. Как и значения стажа остальных пенсионеров - для корректного анализа их необходимо заменить. Будем опираться на среднее время работы мужчин и женщин - 41,5 и 36,5 лет соответственно, или 15147 и 13322 дней. Чтобы избавитьс от пропущенных значений в столбце *total_income*, мы нашли медианное значение для каждого значения в столбце *income_type*. После чего для каждого респоднета заменили пропущенное значение на медианное его типа, тоесть: студент - медианное значение по струднтам, пенсионер - медианное значение по пенсионерам, сотрудник - медианное значение сотрудника и т.д.
P.S. С прошедним 8 марта

### Замена типа данных

In [10]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

**Вывод**

Столбцы: *days_employed* и *total_income* имели тип *float*. Для изменения типа данных мы использовали метод **astype()** для перевода значений в целочисленные

### Обработка дубликатов

In [11]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df.duplicated().sum()
df = df.drop_duplicates()
df.duplicated().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**

Изначально мы нашли 71 дубликата, их количество весьма мало, поэтому можем их просто удалить с помощью функции *drop_duplicates()*

### Лемматизация

In [12]:
def lemm_purpose(text):
    lemmas = m.lemmatize(text)
    text = ' '.join(lemmas)
    return text
df['purpose_lemmas'] = df['purpose'].apply(lemm_purpose)
list_for_count = [string.split(' ') for string in df['purpose_lemmas'].values]
def lemmas_counter(list_for_lemmas):
    purpose_lemmas = []
    for row in list_for_lemmas:
        for col in row:
            purpose_lemmas.append(col)
    return purpose_lemmas
lemmatised_list = lemmas_counter(list_for_count)
Counter(lemmatised_list)

Counter({'покупка': 5897,
         '': 67140,
         'жилье': 4460,
         '\n': 21454,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

**Вывод**

В результате леммации мы можем выделить основные категории целей кредитов: *жилье*, *автомобиль*, *образование*, *свадьба*, *недвижимость*,. Так как, следующий наш шаг это категоризация, мы принимаем решение объединить несколько лемм в одну, так как по смыслу они похожи, а именно: *жилье*, *недвижимость*. Объединим это все в одну лемму *недвижимость*

### Категоризация данных

In [13]:

for row in range(len(df)):
    if "недвижимость" or "жилье" in str(df.loc[row, 'purpose_lemmas']):
        df.loc[row, 'class_lemmas'] = 'недвижимость'
    if "автомобиль" in str(df.loc[row, 'purpose_lemmas']):
        df.loc[row, 'class_lemmas'] = 'автомобиль'
    if "образование" in str(df.loc[row, 'purpose_lemmas']):
        df.loc[row, 'class_lemmas'] = 'образование'
    if "свадьба" in str(df.loc[row, 'purpose_lemmas']):
        df.loc[row, 'class_lemmas'] = 'свадьба'

def income_group(total_income):
    if total_income <= 40000:
        return 'низкий'
    if 40001 <= total_income <= 80000:
        return 'средний'
    if total_income >= 80001:
        return 'высокий'
df['class_income'] = df['total_income'].apply(income_group)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,class_lemmas,class_income
0,1.0,8437.0,42.0,высшее,0.0,женат / замужем,0.0,F,сотрудник,0.0,253875.0,покупка жилья,покупка жилье \n,недвижимость,высокий
1,1.0,4024.0,36.0,среднее,1.0,женат / замужем,0.0,F,сотрудник,0.0,112080.0,приобретение автомобиля,приобретение автомобиль \n,автомобиль,высокий
2,0.0,5623.0,33.0,среднее,1.0,женат / замужем,0.0,M,сотрудник,0.0,145885.0,покупка жилья,покупка жилье \n,недвижимость,высокий
3,3.0,4124.0,32.0,среднее,1.0,женат / замужем,0.0,M,сотрудник,0.0,267628.0,дополнительное образование,дополнительный образование \n,образование,высокий
4,0.0,13322.0,53.0,среднее,1.0,гражданский брак,1.0,F,пенсионер,0.0,158616.0,сыграть свадьбу,сыграть свадьба \n,свадьба,высокий


'**Вывод**

Для удобста ответов на вопросы нашего исселования мы выделили ряд категоризацей нашей таблицы: 
    
* 1) категоризация по цели кредита 
* 2) категоризация по уровню дохода
    
Разделение респондентов по уровню дохода мы сделали следующим образом по данным Росстата средня заработная плата по стране составляет более 40 000 рублей в месяц, следовательно любой доход до это цифры мы принимаем как низких доход. Средний доход мы будем принимать как 2 низких дохода. Все что будет выше это высокий доход.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [14]:
pt_children = df.pivot_table(index = 'children', columns = 'debt', values = 'purpose', aggfunc = ['count'])
df_children = pd.DataFrame(pt_children.to_records())
df_children.columns = ['children', 'debt_0', 'debt_1']
df_children['debt_sum'] = df_children['debt_0'] + df_children['debt_1']
df_children['percent'] = (df_children['debt_1'] / df_children['debt_sum']) * 100
df_children



,children,debt_0,debt_1,debt_sum,percent
0,-1.0,46.0,1.0,47.0,2.127660
1,0.0,13028.0,1063.0,14091.0,7.543822
2,1.0,4364.0,444.0,4808.0,9.234609
3,2.0,1858.0,194.0,2052.0,9.454191
4,3.0,303.0,27.0,330.0,8.181818
5,4.0,37.0,4.0,41.0,9.756098
6,5.0,9.0,NaN,NaN,NaN
7,20.0,68.0,8.0,76.0,10.526316


**Вывод**

Чтобы ответить на поставленный вопрос нам нужно посчитать вероятность клиента стать должником, для этого поделим кол-во должников на общее кол-во. Из полученной нами таблицы мы можем следать следующий вывод: респонденты с детьми имеют больший шанс стать должниками в отличии от респондентов без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [15]:
pt_family = df.pivot_table(index = 'family_status', columns = 'debt', values = 'family_status_id', aggfunc = ['count'])
df_family = pd.DataFrame(pt_family.to_records())
df_family.columns = ['family_status', 'debt_0', 'debt_1']
df_family['debt_sum'] = df_family['debt_0'] + df_family['debt_1']
df_family['percent'] = (df_family['debt_1'] / df_family['debt_sum']) * 100
df_family

,family_status,debt_0,debt_1,debt_sum,percent
0,в разводе,1110,85,1195,7.112971
1,вдовец / вдова,896,63,959,6.569343
2,гражданский брак,3763,388,4151,9.347145
3,женат / замужем,11408,931,12339,7.545182
4,не женат / не замужем,2536,274,2810,9.750890


**Вывод**

Чтобы ответить на поставленный вопрос нам нужно посчитать вероятность клиента стать должником, для этого поделим кол-во должников на общее кол-во. Из полученной нами таблицы мы можем сделать следующий вывод: респонденты с семейным положением "гражданский брак" и "не женат / не замужем" имеют самую высокую вероятность стать должниками. Респонденты с семейным положением "в разводе" и "женат / замужем" имеют примерно равную вероятность стать должниками

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
pt_class_income = df.pivot_table(index = 'class_income', columns = 'debt', values = 'purpose', aggfunc = ['count'])
df_class_income = pd.DataFrame(pt_class_income.to_records())
df_class_income.columns = ['class_income', 'debt_0', 'debt_1']
df_class_income['debt_sum'] = df_class_income['debt_0'] + df_class_income['debt_1']
df_class_income['percent'] = (df_class_income['debt_1'] / df_class_income['debt_sum']) * 100
df_class_income

,class_income,debt_0,debt_1,debt_sum,percent
0,высокий,17611,1567,19178,8.170821
1,низкий,115,10,125,8.000000
2,средний,1987,164,2151,7.624361


**Вывод**

Чтобы ответить на поставленный вопрос нам нужно посчитать вероятность клиента стать должником, для этого поделим кол-во должников на общее кол-во.
    
* Вероятность появления задолженности у клиентов с низким уровнем дохода - **8,0%**
* Вероятность появления задолженности у клиентов с средним уровнем дохода - **7,6%**
* Вероятность появления задолженности у клиентов с высоким уровнем дохода - **8,2%**

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
df.groupby('class_lemmas')['debt'].agg(['count','mean'])
pt_class_lemmas = df.pivot_table(index = 'class_lemmas', columns = 'debt', values = 'purpose', aggfunc = ['count'])
df_class_lemmas = pd.DataFrame(pt_class_lemmas.to_records())
df_class_lemmas.columns = ['class_lemmas', 'debt_0', 'debt_1']
df_class_lemmas['debt_sum'] = df_class_lemmas['debt_0'] + df_class_lemmas['debt_1']
df_class_lemmas['percent'] = (df_class_lemmas['debt_1'] / df_class_lemmas['debt_sum']) * 100
df_class_lemmas

,class_lemmas,debt_0,debt_1,debt_sum,percent
0,автомобиль,3886,400,4286,9.332711
1,недвижимость,10002,777,10779,7.208461
2,образование,3635,367,4002,9.170415
3,свадьба,2130,186,2316,8.031088


**Вывод**

Чтобы ответить на поставленный вопрос нам нужно посчитать вероятность клиента стать должником, для этого поделим кол-во должников на общее кол-во.
    
* Вероятность появления задолженности у клиентов с целью кредита "автомобиль" - **9,4%**
* Ветоятность появления задолженности у клиентов с целью кредита "недвижимость" - **7,2%**
* Ветоятность появления задолженности у клиентов с целью кредита "образование" - **9,2%**
* Вероятность появления задолженности у клиентов с целью кредита " свадьба" - **8,0%**
    </div>

## Шаг 4. Общий вывод

Мы провели анализ зависимости возврата кредита в срок в разных категориях. Клиенты с детьми чаще возвращают кредиты в срок. Клиенты в браке возвращают кредиты в срок почти также как и клиенты без брака. Клиенты с высоким уровнем дохода оказались самой рискогой группой среди категории доход. Цель кредита "недвижимость" увеличивает вероятность невозврата в двое по отношению к остальным целям